In [20]:
import os
import dotenv

dotenv.load_dotenv()

True

In [7]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain

In [8]:


db = SQLDatabase.from_uri("sqlite:///Chinook.db") #being ideal for demos and testing ORM tools
llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [14]:
answer = db_chain.run("How many employees are there?")



> Entering new SQLDatabaseChain chain...
How many employees are there?
SQLQuery:SELECT COUNT(*) FROM "Employee";
SQLResult: [(8,)]
Answer:There are 8 employees.
> Finished chain.


In [15]:
answer

'There are 8 employees.'

# Case 1: Text-to-SQL query

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_sql_query_chain

In [23]:
chain = create_sql_query_chain(ChatOpenAI(temperature=0), db)
response = chain.invoke({"question":"How many employees are there"})
print(response)

SELECT COUNT(*) FROM Employee


In [25]:
answer = db.run(response)

In [30]:
eval(answer)[0][0]

8

# Case 2: Text-to-SQL query to Execution to Summarization

Some research papers have reported good performance when prompting with:

- A CREATE TABLE description for each table, which include column names, their types, etc
- Followed by three example rows in a SELECT statement

In [ ]:
#create_sql_query_chain adopts this the best practice

The query builder can be improved in several ways, such as (but not limited to):

- Customizing database description to your specific use case
- Hardcoding a few examples of questions and their corresponding SQL query in the prompt
- Using a vector database to include dynamic examples that are relevant to the specific user question

# Prompt Template

In [114]:
from langchain.prompts import PromptTemplate

TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer with a Summarization.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}.

Some examples of SQL queries that correspond to questions are:

Question: What is the total number of employees?
SQLQuery: SELECT COUNT(*) FROM Employee

Question: What is the country with the largest number of employees?
SQLQuery: SELECT Country, COUNT(*) AS EmployeeCount FROM Employee GROUP BY Country ORDER BY EmployeeCount DESC LIMIT 1;

Question: {input}"""

CUSTOM_PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=TEMPLATE,
)

In [115]:
# table_names = ['Album', 'Artist', 'Genre', 'MediaType', 'Track', 'Employee', 'Invoice', 'InvoiceLine', 'Playlist', 'PlaylistTrack', 'Customer']

In [116]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db", include_tables=['Employee'], sample_rows_in_table_info=2)

llm = OpenAI(temperature=0, verbose=True)

db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=CUSTOM_PROMPT, verbose=True, use_query_checker=True)

In [100]:
# print(db.table_info)

In [117]:
db_chain.run("What are the available cities")



> Entering new SQLDatabaseChain chain...
What are the available cities
SQLQuery:SELECT DISTINCT City FROM Employee;
SQLResult: [('Edmonton',), ('Calgary',), ('Lethbridge',)]
Answer:The available cities are Edmonton, Calgary, and Lethbridge.
> Finished chain.


'The available cities are Edmonton, Calgary, and Lethbridge.'

In [118]:
db_chain.run("What are the distinct countries in the data")



> Entering new SQLDatabaseChain chain...
What are the distinct countries in the data
SQLQuery:SELECT DISTINCT Country FROM Employee;
SQLResult: [('Canada',)]
Answer:The distinct countries in the data are Canada.
> Finished chain.


'The distinct countries in the data are Canada.'

In [119]:
db_chain.run("Which city has the most employees")



> Entering new SQLDatabaseChain chain...
Which city has the most employees
SQLQuery:SELECT COUNT(*) FROM Employee
SQLResult: [(8,)]
Answer:There are 8 employees.
> Finished chain.


'There are 8 employees.'

In [120]:
db_chain.run("select * from Employee")



> Entering new SQLDatabaseChain chain...
select * from Employee
SQLQuery:SELECT * FROM Employee;
SQLResult: [(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com'), (2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2002-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com'), (3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2002-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com'), (4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2003-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com'), (5, 'Johns

'There are 8 employees in the Employee table.'

In [121]:
db_chain.run("show tables")



> Entering new SQLDatabaseChain chain...
show tables
SQLQuery:SELECT name FROM sqlite_master WHERE type='table';
SQLResult: [('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
Answer:The tables in the database are Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, and Track.
> Finished chain.


'The tables in the database are Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, and Track.'